In [1]:
from twikit import Client
import time
import json
from twikit import TwitterException 
import pandas as pd

# this API requires authentication
f = open('authentication.txt', 'r')
auth = f.read()
f.close()
auth_token = auth.split("\n")

# don't hardcode your email and password into something!!!
# the auth is in gitignore so I won't get hacked
USERNAME = str(auth_token[0])
EMAIL = str(auth_token[1])
PASSWORD = str(auth_token[2])

# Initialize client
client = Client(language='en-US', http2=True)

# Login to the service with provided user credentials
client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

C:\Users\eaesp\AppData\Local\Temp\ipykernel_37468\2204781778.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


{'flow_token': 'g;171365079169825533:-1713650791756:i7XGUAdAqiL6B4BOIfqYgKwQ:14',
 'status': 'success',
 'subtasks': [{'subtask_id': 'LoginSuccessSubtask',
   'open_account': {'user': {'id': 1547081484695216130,
     'id_str': '1547081484695216130',
     'name': 'Eloragh Espie',
     'screen_name': 'EloraghEspie'},
    'next_link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'SuccessExit'},
    'attribution_event': 'login'}},
  {'subtask_id': 'SuccessExit',
   'open_link': {'link': {'link_type': 'subtask',
     'link_id': 'next_link',
     'subtask_id': 'LoginOpenHomeTimeline'}}},
  {'subtask_id': 'LoginOpenHomeTimeline',
   'open_home_timeline': {'next_link': {'link_type': 'abort',
     'link_id': 'next_link'}}}]}

In [2]:
# Twitter LOVES to ban people when they log in repeatedly
# saving the cookies makes sure I don't get banned (often)

client.get_cookies()
client.save_cookies('cookies.json')
with open('cookies.json', 'r', encoding='UTF8') as f:
    client.set_cookies(json.load(f))

In [3]:
def get_user_id(handle):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    return(client.get_user_by_screen_name(handle))

In [4]:
def create_key(user_id, year):
    # load the cookies so you don't login a million times and get banned
    client.load_cookies('cookies.json')
    return(user_id + year)

In [14]:
import time
from math import ceil
import requests
from twikit import TooManyRequests
from twikit.utils import Endpoint

# this is a housekeeping function
# twitter API can throw rate limits
# they're kind of like timeouts
# this function just shows me how much longer I will be in timeout for

def get_limit_reset_time(endpoint: str):
    res = requests.get(
        endpoint,
        headers=client._base_headers,
        cookies=client.get_cookies()
    )
    return ceil(int(res.headers['x-rate-limit-reset']) - time.time())

In [27]:
# timeout check for accessing user IDs
try:
    print(client.get_user_by_screen_name('BarackObama'))
except TooManyRequests:
    reset_time = get_limit_reset_time(Endpoint.USER_TWEETS)
    print(f'rate limit is reset after {reset_time} seconds.')

rate limit is reset after 897 seconds.


In [8]:
data = pd.read_csv("Political Compass Coordinates Data.csv")
df = pd.DataFrame(data)

In [24]:
# turn pandas into list
# there is probably a better way to do this but this works 
i = 0
data_list = []
for row in df.iterrows():
    row_list = df.loc[i, :].values.flatten().tolist()
    data_list.append(row_list)
    i += 1
print(data_list)

[[nan, nan, 'Barack Obama', 'BarackObama', 3, 2, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Joe Biden', 'JoeBiden', 3, 3, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Dennis Kucinich', 'Dennis_Kucinich', -2, -2, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Bill Richardson', 'GovRichardson', 4, 4, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Mike Huckabee', 'GovMikeHuckabee', 6, 6, 'Republican', 2008, 'USA', nan], [nan, nan, 'Hillary Clinton', 'HillaryClinton', 4, 2, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Ron Paul', 'RonPaul', 9, 1, 'Republican', 2008, 'USA', nan], [nan, nan, 'John McCain', 'SenJohnMcCain', 6, 4, 'Republican', 2008, 'USA', nan], [nan, nan, 'Newt Gingrich', 'newtgingrich', 8, 7, 'Republican', 2008, 'USA', nan], [nan, nan, 'Mike Gravel', 'MikeGravel_US', 8, -2, 'Democratic', 2008, 'USA', nan], [nan, nan, 'Alan Keyes', 'loyaltoliberty', 6, 8, 'Republican', 2008, 'USA', nan], [nan, nan, 'Mitt Romney', 'MittRomney', 7, 8, 'Republican', 2008, 'USA', nan], [nan, nan, '

In [25]:
from twikit import UserUnavailable
from twikit import UserNotFound

for item in data_list:
    # pull IDs and insert them back into the list
    handle = item[3]
    print(handle)
    try:
        user_id = get_user_id(handle)
        item.insert(1, user_id.id)
        print(f'Success: {handle}, {user_id}')
    # if user is inaccessible this will keep the loop from breaking
    except UserUnavailable:
        print(f"User {handle} unavailable")
    except UserNotFound:
        print(f'User not found {handle}')
    time.sleep(5)  # cooldown so we don't get banned

BarackObama
Success: BarackObama, <User id="813286">
JoeBiden
Success: JoeBiden, <User id="939091">
Dennis_Kucinich
Success: Dennis_Kucinich, <User id="15022633">
GovRichardson
Success: GovRichardson, <User id="31428685">
GovMikeHuckabee
Success: GovMikeHuckabee, <User id="15416505">
HillaryClinton
Success: HillaryClinton, <User id="1339835893">
RonPaul
Success: RonPaul, <User id="287413569">
SenJohnMcCain
Success: SenJohnMcCain, <User id="19394188">
newtgingrich
Success: newtgingrich, <User id="20713061">
MikeGravel_US
Success: MikeGravel_US, <User id="752638969095999489">
loyaltoliberty
Success: loyaltoliberty, <User id="19637821">
MittRomney
Success: MittRomney, <User id="50055701">
fredthompson
Success: fredthompson, <User id="2704951">
SenChrisDodd
Success: SenChrisDodd, <User id="16317406">
RudyGiuliani
Success: RudyGiuliani, <User id="770781940341288960">
RalphNader
Success: RalphNader, <User id="77314692">
ttancredo
Success: ttancredo, <User id="64534908">
DrJillStein


TooManyRequests: status: 429, message: "Rate limit exceeded
"